In [ ]:
# APAGAR MARCAS

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# ENV

config = dotenv_values('.env')
url_brands = config['VTEX_URL_B2C'] + '/api/catalog_system/pvt/brand/list'
url_brands_delete = config['VTEX_URL_B2C'] + '/api/catalog/pvt/brand'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

# FUNCAO QUE CHAMA A LISTA DE MARCAS NO SITE B2C

def list_brand(url_brands, headers, data):
  try:
    response = requests.get(url_brands, headers=headers)
    response.raise_for_status()
    if(response.status_code == 200 and len(json.loads(response.text)) > 0):
      return { 'count': len(json.loads(response.text)), 'data': json.loads(response.text) }
    elif (response.status_code == 200 and len(json.loads(response.text)) == 0):
      print('Sem marcas para exibir')
    elif (response.status_code != 200):
      list_brand()
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar lista de marcas no site Kookabu\n')
    print(e.response.text)

# FUNCAO QUE DELETA A MARCA E RETORNA OK OU NOK PARA RELATORIO

def delete_brand(url_brands, headers, data):
  try:
    response = requests.delete(url_brands + '/' + str(data['id']), headers=headers)
    response.raise_for_status()
    if(response.status_code == 200):
      print('Marca {} - ID {} - APAGADA'.format(data['name'], data['id']))
      return { 'id': data['id'], 'marca': data['name'], 'status': 'OK' }
    elif (response.status_code != 200):
      delete_brand(url_brands, headers, data)
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar apagar a Marca {} - ID {} no site Kookabu\n'.format(data['name'], data['id']))
    print(e.response.text)
    return { 'id': data['id'], 'marca': data['name'], 'status': 'NOK' }

# ITERA AS MARCAS, CHAMA AS FUNCOES E AGRUPA DADOS PARA RELATORIO

brand_report = []
count = 0
content = list_brand(url_brands, headers, {})
for brand in content['data']:
  if(brand['name'] == config['KEEP_BRAND_NAME'] and str(brand['id']) != config['KEEP_BRAND_ID']):
    data = delete_brand(url_brands_delete, headers, brand)
    if(data['status'] != 'NOK'):
      brand_report.append(data)
      count += 1
    elif(data['status'] != 'OK'):
      brand_report.append(data)

# REPORT

print('CONTAGEM MARCAS APAGADAS CONFORME CRITERIO CONFIGURADO (MARCA {} - ID {})\n'.format(config['KEEP_BRAND_NAME'], config['KEEP_BRAND_ID']))
print('LIDOS -> {}\n'.format(content['count']))
print('APAGADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-marcas-desativadas.xlsx'), engine='xlsxwriter')
pd.DataFrame(brand_report).to_excel(writer, sheet_name='brand_report')
writer.save()

In [ ]:
# ATUALIZAR MARCA DO PRODUTO 

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# ENV

def product_interface(url_products, headers, data, interface):
  if(interface == 'get'):
    try:
      response = requests.get(url_products + '/' + str(data), headers=headers)
      response.raise_for_status()
      if(response.status_code == 200):
        return json.loads(response.text)
      elif (response.status_code != 200):
        product_interface(url_products, headers, {}, 'get')
    except requests.exceptions.HTTPError as e:
      print('Erro ao chamar produto')
      print('\n{}'.format(e.response.text))
  elif(interface == 'put'):
    try:
      response = requests.put(url_products + '/' + str(data['id']), headers=headers, data=data['data'])
      response.raise_for_status()
      if(response.status_code == 200):
        return { 'productId': data['id'], 'status': 'OK', 'mensagem': 'OK' }
      elif (response.status_code != 200):
        product_interface(url_products, headers, data, 'put')
    except requests.exceptions.HTTPError as e:
      print('Erro ao atualizar o produto')
      print('\n{}'.format(e.response.text))
      return { 'productId': data['id'], 'status': 'NOK', 'mensagem': e.response.text }

config = dotenv_values('.env')
url_products = config['VTEX_URL_B2C'] + '/api/catalog/pvt/product'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

df = pd.read_csv(os.getcwd() + '/specs/inputs/products.csv')

products_to_commit = []
count = 0
for product in df.itertuples():
  data = product_interface(url_products, headers, product.productId, 'get')
  payload = {
   'Name': data['Name'],
   'DepartmentId': data['DepartmentId'],
   'CategoryId': data['CategoryId'],
   'BrandId': 2003171,
   'LinkId': data['LinkId'],
   'RefId': data['RefId'],
   'IsVisible': data['IsVisible'],
   'Description': data['Description'],
   'DescriptionShort': data['DescriptionShort'],
   'ReleaseDate': data['ReleaseDate'],
   'KeyWords': data['KeyWords'],
   'Title': data['Title'],
   'IsActive': data['IsActive'],
   'TaxCode': data['TaxCode'],
   'MetaTagDescription': data['MetaTagDescription'],
   'SupplierId': data['SupplierId'],
   'ShowWithoutStock': data['ShowWithoutStock'],
   'AdWordsRemarketingCode': data['AdWordsRemarketingCode'],
   'LomadeeCampaignCode': data['AdWordsRemarketingCode'],
   'Score': data['Score']
  }
  data = product_interface(url_products, headers, {'id': data['Id'], 'data': payload}, 'put')
  if(data['status'] != 'NOK'):
    products_to_commit.append(data)
    count += 1
  elif(data['status'] != 'OK'):
    products_to_commit.append(data)

print('CONTAGEM DE PRODUTOS PARA ATUALIZAR -> {}\n'.format(len(df)))
print('PROCESSADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-produtos-atualizados.xlsx'), engine='xlsxwriter')
pd.DataFrame(products_to_commit).to_excel(writer, sheet_name='product_report')
writer.save()